# **CHURN PREDICTION**

## **Import necessary libraries and methods**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder
from scipy.stats import spearmanr
import re

PERIOD_DICT = {}
PLAN_DICT = {}
PACK_DICT = {}
REGION_DICT = {}

## **Dataset preparation**

### **Get datasets**

In [ ]:
data = pd.read_csv('Train.csv')
zindi_data = pd.read_csv('Test.csv')
data.head(10)

Here we can notice that **TOP_PACK** colunm may contain more information. So, feature engineering with this colum may improve dataset

In [ ]:
data.columns

In [ ]:
data.shape

### **Cheсk missing values**

In [ ]:
def perc_missing(df):
    '''prints out columns with missing values with its %'''
    for col in df.columns:
        pct = df[col].isna().mean() * 100
        if (pct != 0):
            print('{} => {}%'.format(col, round(pct, 2)))
    
perc_missing(data)

From the calculations we can see that the percentage of empty values in the attributes **TIGO, ZONE1, ZONE2** is more than 50%. If we replace these values with the average, we will shift the data and it will be implausible. If we remove rows with empty values, we will lose a lot of information, so these attributes should be removed from the dataset. But, to be sure we can check performance of model including any combination of these features.

Next plot Nan values using heatmap for both train and test datasets

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(data.isnull(), yticklabels=False, cmap='viridis', cbar=False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(zindi_data.isnull(), yticklabels=False, cmap='viridis', cbar=False)

Now check the balance of **CHURN** values 

In [ ]:
print(data['CHURN'].value_counts())
plt.figure(figsize=(10,5))
data['CHURN'].value_counts(normalize=True).plot(kind='bar')
plt.ylabel('counts')
plt.xlabel('Churn')

As we can see on a plot, '0' values are much more frequent. \
To balance 0 and 1 values in CHURN we will drop all the rows, that have at least **k** Nan value with 0 in CHURN. But **thresh** parameter should be tuned experementally. 

In [ ]:
# data_zero = data.loc[data.CHURN == 0]
# data_one = data.loc[data.CHURN == 1]
# data_zero=data_zero.dropna(thresh = 11)
# data = pd.concat([data_zero, data_one])
# data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.nunique()

_____________________________________________________________________________________________________________________

### **Feature engineering** 
We need to :
1. convert values in categorical features into numeric
2. replace all Nan values 
3. remove unnecessary features
4. renerate new features if possible 

Fistly, all the values in **user_id** field are different, and the **MRG** has only one value. This means that these features have absolutely no effect on the target value, so they should be removed.

Let's separate features for categorical and numerical.

In [ ]:
cat_df = data.select_dtypes(include=['object'])
num_df = data.select_dtypes(exclude=['object'])

In [ ]:
cat_df.REGION.fillna('other', inplace=True)
cat_df.REGION.replace(REGION_DICT, inplace=True)

cat_df.TENURE.fillna('other', inplace=True)
cat_df.TENURE.replace(TENURE_DICT, inplace=True)

In [ ]:
df = pd.DataFrame()
df['TENURE'] = cat_df['TENURE']
df['REGULARITY'] = num_df['REGULARITY']
df['CHURN'] = num_df['CHURN']
df['DATA_VOLUME'] = num_df['DATA_VOLUME']
print(df.shape[0])
df = df.iloc[:df.shape[0]//2, :]

In [ ]:
imputer = KNNImputer(n_neighbors=1)
res = pd.DataFrame(imputer.fit_transform(df), columns=['TENURE', 'REGULARITY', 'CHURN', 'DATA_VOLUME'])

In [ ]:
res

In [ ]:
cat_df.head(5)

In [ ]:
num_df.head(5)

_____________________________________________________________________________________________________________________

### Numerical features

To deal with with missing values we will impute them. Dropping feature isn’t recommended because we can lose information. \
But we can use different imputer, such as **SimpleImputer**, **KNNImputer** or **DataWig** 

_____________________________________________________________________________________________________________________

### Categorical features

From the whole dataset we could extract three categorical featured : **REGION**, **TENURE** and **TOP_PACK**

**REGION** column includes 14 unique region names and NaN values

In [ ]:
cat_df['REGION'].value_counts()

We can create REGION_DICT that will be used for replacing all **REGION**s with int values. But to be sure that replacing will be implemented correctly, we need to check whether test dataset contain any additional **REGION** values. 

In [ ]:
all_regions = pd.concat([cat_df['REGION'], zindi_data['REGION']])
all_regions = list(all_regions.unique())
all_regions[1] = 'other'

counter = 0
for reg in all_regions:
    REGION_DICT[reg] = counter
    counter+=1

Next point - **TENURE** column. \
Seems like it may takes only 8 categories. That give us opportunity to convert it in numeric values manually. 

In [ ]:
cat_df['TENURE'].value_counts()

In [ ]:
TENURE_DICT = {'D 3-6 month': 4.5, 
                'E 6-9 month': 7.5,
                'F 9-12 month': 10.5,
                'G 12-15 month': 13.5,
                'H 15-18 month': 16.5,
                'I 18-21 month': 19.5,
                'J 21-24 month': 22.5,
                'K > 24 month': 25
                }

As **TOP_PACK** contain pretty a lot of text information, we can extract some parts to creat new features for prediction model. Such as **PRICE**, **PERIOD** and **PLAN** 

Get all **TOP_PACK**s from train and test datasets to create separate dataframe, build new features and replace str values with SimpleEncoder. 

To extract information from **TOP_PACK** we can use RegEx \
Create separate functions :

In [ ]:
def reg_expression_period(text):
    text = str(text)
    period_numeric = re.findall(r"\d{1,2}[d|h]", text)
    if len(period_numeric) > 0:
        return period_numeric[-1]
    period_char = re.findall(r"(weekly|monthly|daily|oneday|30_days|night)", text)
    if len(period_char) > 0:
        return period_char[-1]
    return 'idk'


def reg_expression_price(text):
    text = str(text)
    period_numeric = re.findall(r"[0-9]+f", text.replace(' ', ''))
    if len(period_numeric) > 0:
        return float(period_numeric[0][:-1])
    else:
        return -100
    
    
def reg_expression_pack(text):
    text = str(text)
    plan = re.findall(r"(mixt|onnet|data|pilot|cvm|allnet|wifi|internat|twter)", text.replace('-','').replace(' ', ''))
    if len(plan) > 0:
        return plan[0]
    else:
        return 'idk'

Create *top_pack* dataframe and complete all manipulations with **TOP_PACK** column :

In [ ]:
all_packs = pd.concat([cat_df['TOP_PACK'], zindi_data['TOP_PACK']])
all_packs = all_packs.unique()

top_pack = pd.DataFrame()
top_pack['TOP_PACK'] = pd.Series(all_packs).apply(str)
top_pack['TOP_PACK'].append(pd.Series(['other']))
top_pack['lower'] = top_pack['TOP_PACK'].str.lower()

top_pack['TOP_PACK_int'] = top_pack['TOP_PACK']

top_pack['PERIOD'] = top_pack['lower'].apply(reg_expression_period)
top_pack['PRICE'] = top_pack['lower'].apply(reg_expression_price)
top_pack['PLAN'] = top_pack['lower'].apply(reg_expression_pack)
top_pack.drop('lower', axis=1, inplace=True)

LE = LabelEncoder()
top_pack['TOP_PACK_int'] = LE.fit_transform(top_pack['TOP_PACK_int'])
top_pack['PERIOD'] = LE.fit_transform(top_pack['PERIOD'])
top_pack['PLAN'] = LE.fit_transform(top_pack['PLAN'])

Now we have separate dataframe with additional features. \
We can concat this dataframe with our train and test dataset.

_____________________________________________________________________________________________________________________

Now we can bring together all manipulation with datasets into a function.

In [ ]:
def data_preparation(dataset):
    global top_pack, REGION_DICT, TENURE_DICT
    dataset.drop(['user_id', 'MRG'], inplace=True, axis=1)  
        
    #Separating our features into numerical and categorical
    cat_df = dataset.select_dtypes(include=['object'])
    num_df = dataset.select_dtypes(exclude=['object'])
       
    #Replace REGION and TENURE values
    cat_df.REGION.fillna('other', inplace=True)
    cat_df.REGION.replace(REGION_DICT, inplace=True)
    
    cat_df.TENURE.fillna('other', inplace=True)
    cat_df.TENURE.replace(TENURE_DICT, inplace=True)

    # Add PERIOD, PRICE, PLAN and replace  TOP_PACK values
    cat_df['TOP_PACK'].fillna('other', inplace=True)
    cat_df = pd.merge(cat_df, top_pack, how="left", on='TOP_PACK')
    cat_df.drop('TOP_PACK', axis=1, inplace=True)
    cat_df.rename(columns={'TOP_PACK_int': 'TOP_PACK'}, inplace=True)

    #Filling numeric columns with mean values

    # num_df['DATA_VOLUME'] = df['DATA_VOLUME']

    # imr = SimpleImputer(missing_values=np.nan, strategy='mean')
    # num_columns = num_df.columns
    # for i in num_columns:
    #     num_df[[f'{i}']] = imr.fit_transform(num_df[[f'{i}']])
    
    #Ending columns convertation and cleaning we need to put together parts of dataset
    dataset[num_df.columns] = num_df
    dataset[cat_df.columns] = cat_df

    return dataset

Finnaly, we can clean up our dataset 

In [ ]:
data = data_preparation(data)

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

### Correlation Test
Spearman’s Rank Correlation. The function takes two real-valued samples as arguments and returns both the correlation coefficient in the range between -1 and 1 and the p-value for interpreting the significance of the coefficient. This statistical method quantifies the degree to which ranked variables are associated by a monotonic function, meaning an increasing or decreasing relationship.

In [ ]:
data1 = data['CHURN']
for column in data.columns:
    data2 = data[f'{column}']
    stat, p = spearmanr(data1, data2)
    if p > 0.05:
        print(f'{column} and CHURN probably independent')
    else:
        print(f'{column} and CHURN probably dependent')
    print()

### Plot final 0 and 1 value distribution in CHURN

In [ ]:
print(data['CHURN'].value_counts())
plt.figure(figsize=(10,5))
data['CHURN'].value_counts(normalize=True).plot(kind='bar')
plt.ylabel('counts')
plt.xlabel('Churn')

Now dataset is ready for classification model.

_____________________________________________________________________________________________________________________

# **Machine Learning part**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import optuna  # pip install optuna
from sklearn.metrics import mean_squared_error

import lightgbm
from sklearn.metrics import roc_curve, roc_auc_score


Split dataset into three parts : train - 80%, test - 20%.

In [ ]:
y = data.loc[:, 'CHURN']
x = data.drop('CHURN', axis=1)

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state=1)

print(f'Train size - {X_train.shape[0]}')

print(f'Test size - {X_test.shape[0]}')

In [ ]:
data

### Initialize LGBMClassifier

In [ ]:
def objective(trial,X,y):
    
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=1)
    param = {
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 1000, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 1000, step=50),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95, step=0.1),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
        "feature_fraction", 0.2, 0.95, step=0.1
        )
    }
    model = lightgbm.LGBMRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse



study = optuna.create_study(direction='minimize')
func = lambda trial: objective(trial, x, y)
study.optimize(func, n_trials=20)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
best_params = study.best_trial.params
best_params

In [ ]:
best_params['n_estimators']

### Model initialization

In [ ]:
clf = lightgbm.LGBMClassifier(n_estimators=best_params['n_estimators'], 
                              learning_rate=best_params['learning_rate'], 
                              num_leaves=best_params['num_leaves'], 
                              max_depth=best_params['max_depth'], 
                              min_data_in_leaf=best_params['min_data_in_leaf'], 
                              max_bin=best_params['max_bin'], 
                              lambda_l1=best_params['lambda_l1'], 
                              lambda_l2=best_params['lambda_l2'], 
                              min_gain_to_split=best_params['min_gain_to_split'], 
                              bagging_fraction=best_params['bagging_fraction'], 
                              bagging_freq=best_params['bagging_freq'], 
                              feature_fraction=best_params['feature_fraction'])

### Launch learning process

In [ ]:
randmodel = clf.fit(X_train,y_train)
randpred = randmodel.predict(X_test)

### Results

In [ ]:
print("Accuracy")
print(accuracy_score(y_test, randpred))

In [ ]:
print("Precision")
precision_score(y_test, randpred)

In [ ]:
print("Recall")
recall_score(y_test, randpred)

Draw confusion matrix in persantage ratio

In [ ]:
conf = confusion_matrix(y_test, randpred)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                conf.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     conf.flatten()/np.sum(conf)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(conf, annot=labels, fmt='', cmap='Greens')


### Plot AUC ROC curve and calculate roc_auc_score

In [ ]:
rp = randmodel.predict_proba(X_test)[:, 1]

false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, rp)

print('roc_auc_score for lightGBM: ', roc_auc_score(y_test, rp))

plt.subplots(1, figsize=(10,10))
plt.title('Receiver Operating Characteristic - lightGBM')
plt.plot(false_positive_rate1, true_positive_rate1)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

## For competition:

In [ ]:
submission = pd.DataFrame()
submission["user_id"] = zindi_data["user_id"]

zindi_data = data_preparation(zindi_data)
subpred = randmodel.predict_proba(zindi_data)
submission["CHURN"] = subpred[:, 1]
submission.head()
submission.to_csv('starter_code_submission.csv', index=False)